## Introduction to AWS ParallelCluster

This is a shortened version. Steps to prepare (before and after) for the ParallelCluster are coded in pcluster-athena.py. 

#### Before: 
- Create ssh key
- Create or use existing default VPC
- Create an S3 bucket for config, post install, sbatch, Athena++ initial condition files
- Create or use exsiting MySQL RDS database for Slurm accounting
- Update VPC security group to allow traffic to MySQL RDS
- Create a secret for RDS in AWS Secret Manager
- Upload post install script to S3
- Fill the ParallelCluster config template with values of region, VPC_ID, SUBNET_ID, KEY_NAME, POST_INSTALLSCRIPT_LOCATION and POST_INSALL_SCRIPT_ARGS 
- Upload the config file to S3

#### After
- Update VPC security group attached to the headnode to open port 8082 to this notebook

#### Note: 


In [ ]:
import boto3
import botocore
import json
import time
import os
import sys
import base64
import docker
import pandas as pd
import importlib
import project_path # path to helper methods
from lib import workshop
from botocore.exceptions import ClientError
from IPython.display import HTML, display

#sys.path.insert(0, '.')
import pcluster_athena
importlib.reload(pcluster_athena)


# unique name of the pcluster
pcluster_name = 'myPC5c'
config_name = "config"
post_install_script_prefix = "scripts/pcluster_post_install.sh"

# Graviton cluster
#pcluster_name = 'myPC6g'
#config_name="config-c6g"
#post_install_script_prefix = "scripts/pcluster_post_install-6g.sh"


In [ ]:
# this is used during developemnt, to reload the module after a change in the module
try:
    del sys.modules['pcluster_athena']
except:
    #ignore if the module is not loaded
    print('Module not loaded, ignore')
    
from pcluster_athena import PClusterHelper
# create the cluster - # You can rerun the rest of the notebook again with no harm. There are checks in place for existing resoources. 
pcluster_helper = PClusterHelper(pcluster_name, config_name, post_install_script_prefix)


## Create the parallel cluster
In this shortened version, we will run the cluster creation in the background, using the PClusterHelper class. 

The process will take up to 30 minutes. The two steps that take longer than the rest are the creation of a MySQL RDS instance, and the cluster itself. 

If you want to see how each step is done, please use the "pcluster-athena++" notebook in the same directory.
 

In [ ]:

pcluster_helper.create_before()

# can not properly capture the status output of the command line in the notebook from within the helper class. 
# So broke up the creation into before , create and after
# this process will take up to 30 minutes - libhdf takes a long time to compile and install
!pcluster create $pcluster_helper.pcluster_name -nr -c build/$config_name

pcluster_helper.create_after()


In [ ]:

pcluster_status = !pcluster status $pcluster_name

# Grab the IP of the head node, on where Slurm REST endpoint runs. The returned IP is a private IP of the head node. Make sure your SageMaker notebook
# is created in the same VPC (default VPC)
slurm_host = pcluster_status.grep('MasterPrivateIP').s.split()[1]

print(slurm_host)

## Integrate with Slurm REST API running on the head node

SLURM REST is currently running on the headnode. The JWT token is stored in AWS Secret Manager from the head node. You will need that JWT token in the header of all your REST API requests. 

Don't forget to add secretsmanager:GetSecretValue permission to the sagemaker execution role that runs this notebook

### Inspect the Slurm REST API Schema

We will start by examing the Slurm REST API schema


In [ ]:
import requests
import json

slurm_openapi_ep = 'http://'+slurm_host+':8082/openapi/v3'
slurm_rest_base='http://'+slurm_host+':8082/slurm/v0.0.35'

_, get_headers = pcluster_helper.update_header_token()

resp_api = requests.get(slurm_openapi_ep, headers=get_headers)
print(resp_api)

if resp_api.status_code != 200:
    # This means something went wrong.
    print("Error" , resp_api.status_code)

with open('build/slurm_api.json', 'w') as outfile:
    json.dump(resp_api.json(), outfile)

print(json.dumps(resp_api.json(), indent=2))


### Use REST API callls to interact with ParallelCluster

Then we will make direct REST API requests to retrieve the partitions in response

If you get server errors, you can login to the head-node and check the system logs of "slurmrestd", which is running as a service. 


In [ ]:

partition_info = ["name", "nodes", "nodes_online", "total_cpus", "total_nodes"]

##### This works as well, 
# update header in case the token has expired
_, get_headers = pcluster_helper.update_header_token()

##### call REST API directly
slurm_partitions_url= slurm_rest_base+'/partitions/'
partitions = pcluster_helper.get_response_as_json(slurm_partitions_url)
#print(partitions['partitions'])
#20.02.4 returns a dict, not an array
pcluster_helper.print_table_from_dict(partition_info, partitions['partitions'])

# newer slurmrest return proper array
# print_table_from_json_array(partition_info, [partitions['partitions']['q1'], partitions['partitions']['q2']] )


### Submit a job
The slurm_rest_api_client job submit function doesn't include the "script" parameter. We will have to use the REST API Post directly. 

The body of the post should be like this.  

```
{"job": {"account": "test", "ntasks": 20, "name": "test18.1", "nodes": [2, 4],
"current_working_directory": "/tmp/", "environment": {"PATH": "/bin:/usr/bin/:/usr/local/bin/","LD_LIBRARY_PATH":
"/lib/:/lib64/:/usr/local/lib"} }, "script": "#!/bin/bash\necho it works"}
```
When the job is submitted through REST API, it will run as the user "slurm". That's what the work directory "/shared/tmp" should be owned by "slurm:slurm", which is done in the post_install script. 

fetch_and_run.sh will fetch the sbatch script and the input file from S3 and put them in /shared/tmp




### Program batch script, input and output files

To share the pcluster among different users and make sure users can only access their own input and output files, we will use user's ow S3 buckets for input and output files.

The job will be running on the ParallelCluster under /efs/tmp (for example) through a fatch (from the S3 bucket) and run script and the output will be stored in the same bucket under "output" path. 


In [ ]:

################## Change these 
# Where the batch script, input file, output files are uploaded to S3
#cluster_instance_type = 'c6gn.2xlarge'
#simulation_name = "orz-512x512"

cluster_instance_type = 'c5n.2xlarge'
simulation_name = "orz-512x512"

# for c5n.18xlarge without HyperThreading, the number of cores is 32 - change this accordingly. 
#num_cores_per_node = 32
#partition = "q1"
# for c5n.2xlarge without HyperThreading, the number of cores is 4 - change this accordingly. 
num_cores_per_node = 4
partition = "q3"
# for c5n.2xlarge wit HyperThreading, the number of cores is 4 - change this accordingly. 
#num_cores_per_node = 8
#partition = "q2"


job_name = f"{simulation_name}-{cluster_instance_type}"
# fake account_name 
account_name = f"dept-2d" 


# turn on/off EFA support in the script
use_efa="NO"
################# END change

# prefix in S3
my_prefix = "athena/"+job_name

# template files for input and batch script
input_file_ini = "config/athinput_orszag_tang.ini"
batch_file_ini = "config/batch_athena_sh.ini"

# actual input and batch script files
input_file = "athinput_orszag_tang.input"
batch_file = "batch_athena.sh"
    
################## Begin ###################################
# Mesh/Meshblock parameters
# nx1,nx2,nx3 - number of zones in x,y,z
# mbx1, mbx2, mbx3 - meshblock size 
# nx1/mbx1 X nx2/mbx2 X nx3/mbx3 = number of meshblocks - this should be the number of cores you are running the simulation on 
# e.g. mesh 100 X 100 X 100 with meshsize 50 X 50 X 50 will yield 2X2X2 = 8 blocks, run this on a cluster with 8 cores 
# 

#Mesh - actual domain of the problem 
# 512X512X512 cells with 64x64x64 meshblock - will have 8X8X8 = 512 meshblocks - if running on 32 cores/node, will need 
# 512/32=16 nodes
nx1=512
nx2=512
nx3=1

#Meshblock - each meshblock size - not too big 
mbnx1=64
mbnx2=64
mbnx3=1


num_of_threads = 1

################# END ####################################

#Make sure the mesh is divisible by meshblock size
# e.g. num_blocks = (512/64)*(512/64)*(512/64) = 8 x 8 x 8 = 512
num_blocks = (nx1/mbnx1)*(nx2/mbnx2)*(nx3/mbnx3)

###
# Batch file parameters
# num_nodes should be less than or equal to the max number of nodes in your cluster
# num_tasks_per_node should be less than or equal to the max number of nodes in your cluster 
# e.g. 512 meshblocks / 32 core/node * 1 core/meshblock = 16 nodes -  c5n.18xlarge
num_nodes = int(num_blocks/num_cores_per_node)

num_tasks_per_node = num_blocks/num_nodes/num_of_threads
cpus_per_task = num_of_threads


#This is where the program is installed on the cluster
exe_path = "/shared/athena-public-version/bin/athena"
#This is where the program is going to run on the cluster
work_dir = '/shared/tmp/'+job_name
ph = { '${nx1}': str(nx1), 
       '${nx2}': str(nx2),
       '${nx3}': str(nx3),
       '${mbnx1}': str(mbnx1),
       '${mbnx2}': str(mbnx2),
       '${mbnx3}': str(mbnx3), 
       '${num_of_threads}' : str(num_of_threads)}
pcluster_helper.template_to_file(input_file_ini, 'build/'+input_file, ph)

ph = {'${nodes}': str(num_nodes),
      '${ntasks-per-node}': str(int(num_tasks_per_node)),
      '${cpus-per-task}': str(cpus_per_task),
      '${account}': account_name,
      '${partition}': partition,
      '${job-name}': job_name,
      '${EXE_PATH}': exe_path,
      '${WORK_DIR}': work_dir,
      '${input-file}': input_file,
      '${BUCKET_NAME}': pcluster_helper.my_bucket_name,
      '${PREFIX}': my_prefix,
      '${USE_EFA}': use_efa,
      '${OUTPUT_FOLDER}': "output/",
      '${NUM_OF_THREADS}' : str(num_of_threads)}
pcluster_helper.template_to_file(batch_file_ini, 'build/'+batch_file, ph)

# upload to S3 for use later
pcluster_helper.upload_athena_files(input_file, batch_file, my_prefix)



In [ ]:
job_script = "#!/bin/bash\n/shared/tmp/fetch_and_run.sh {} {} {} {} {}".format(pcluster_helper.my_bucket_name, my_prefix, input_file, batch_file, job_name)

slurm_job_submit_base=slurm_rest_base+'/job/submit'

#in order to use Slurm REST to submit jobs, you need to have the working directory permission set to nobody:nobody. in this case /efs/tmp
data = {'job':{ 'account': account_name, 'partition': partition, 'name': job_name, 'current_working_directory':'/shared/tmp/', 'environment': {"PATH": "/bin:/usr/bin/:/usr/local/bin/:/opt/slurm/bin:/opt/amazon/openmpi/bin","LD_LIBRARY_PATH":
"/lib/:/lib64/:/usr/local/lib:/opt/slurm/lib:/opt/slurm/lib64"}}, 'script':job_script}

###
# This job submission will generate two jobs , the job_id returned in the response is for the bash job itself. the sbatch will be the job_id+1 run subsequently.
#
resp_job_submit = pcluster_helper.post_response_as_json(slurm_job_submit_base, data=json.dumps(data))


print(resp_job_submit)


### List recent jobs

In [ ]:
# get the list of all the jobs immediately after the previous step. This should return two running jobs. 
slurm_jobs_base=slurm_rest_base+'/jobs'

jobs = pcluster_helper.get_response_as_json(slurm_jobs_base)
# print(jobs)
jobs_headers = [ 'job_id', 'job_state', 'account', 'batch_host', 'nodes', 'cluster', 'partition', 'current_working_directory']

# newer version of slurm 
#print_table_from_json_array(jobs_headers, jobs['jobs'])
pcluster_helper.print_table_from_json_array(jobs_headers, jobs)
                   

A mediu resolution simulation will run about ten minutes, plus the time for the cluster to spin up. Wait till the job finishes running then move to the next sections

# Visualize Athena++ Simulation Results
Now we are going to use the python library comes with Athena++ to read and visualize the simulation results. Simulation data was saved in s3://<bucketname>/athema/$job_name/output folder. 

Import the hdf python code that came with Athena++ and copy the data to local file system. 

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
import h5py

#Do this once. clone the athena++ source code , and the hdf5 python package we need is under vis/python folder

if not os.path.isdir('athena-public-version'):
    !git clone https://github.com/PrincetonUniversity/athena-public-version
else:
    print("Athena++ code already cloned, skip")
    
sys.path.insert(0, 'athena-public-version/vis/python')
import athena_read

In [ ]:
data_folder=job_name+'/output'
output_folder = pcluster_helper.my_bucket_name+'/athena/'+data_folder

if not os.path.isdir(job_name):
    !mkdir -p $job_name
else:
    !rm -rf $job_name/*

!aws s3 cp s3://$output_folder/ ./$data_folder/ --recursive


### Display the hst data
History data shows the overs all parameter changes over time. The time interval can be different from that of the hdf5 files.

In OrszagTang simulations, the variables in the hst files are 'time', 'dt', 'mass', '1-mom', '2-mom', '3-mom', '1-KE', '2-KE', '3-KE', 'tot-E', '1-ME', '2-ME', '3-ME'

All the variables a

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

hst = athena_read.hst(data_folder+'/OrszagTang.hst')

# cannot use this reliably because hst and hdf can have different number of time steps. In this case,we have the same number of steps
num_timesteps = len(hst['time'])

print(hst.keys())

plt.plot(hst['time'], hst['dt'])


## Reading HDF5 data files 

The hdf5 data files contain all variables inside all meshblocks. There are some merging and calculating work to be done before we can visualizing the result. Fortunately ,Athena++ vis/hdf package takes care of the hard part. 


In [ ]:
# Let's example the content of the hdf files

f = h5py.File(data_folder+'/OrszagTang.out2.00001.athdf', 'r')
# variable lists <KeysViewHDF5 ['B', 'Levels', 'LogicalLocations', 'prim', 'x1f', 'x1v', 'x2f', 'x2v', 'x3f', 'x3v']>
print(f.keys())

#<HDF5 dataset "B": shape (3, 512, 64, 64, 64), type "<f4"> 
print(f['prim'])

### Simulation result data 

Raw athdf data has the following keys
<KeysViewHDF5 ['B', 'Levels', 'LogicalLocations', 'prim', 'x1f', 'x1v', 'x2f', 'x2v', 'x3f', 'x3v']>

After athena_read.athdf() call, the result contains keys, which can be used as the field name
['Coordinates', 'DatasetNames', 'MaxLevel', 'MeshBlockSize', 'NumCycles', 'NumMeshBlocks', 'NumVariables', 'RootGridSize', 'RootGridX1', 'RootGridX2', 'RootGridX3', 'Time', 'VariableNames', 'x1f', 'x1v', 'x2f', 'x2v', 'x3f', 'x3v', 'rho', 'press', 'vel1', 'vel2', 'vel3', 'Bcc1', 'Bcc2', 'Bcc3']


In [ ]:
def process_athdf(filename, num_step):
    print("Processing ", filename)
    athdf = athena_read.athdf(filename)
    return athdf

# extract list of fields and take a slice in one dimension, dimension can be 'x', 'y', 'z'
def read_all_timestep (data_file_name_template, num_steps, field_names, slice_number, dimension):

    if not dimension in ['x', 'y', 'z']:
        print("dimension can only be 'x/y/z'")
        return
    
    # would ideally process all time steps together and store themn in memory. However, they are too big, will have to trade time for memory 
    result = {}
    for f in field_names:
        result[f] = list()
        
    for i in range(num_steps):
        fn = data_file_name_template.format(str(i).zfill(5))
        athdf = process_athdf(fn, i)
        for f in field_names:
            if dimension == 'x':
                result[f].append(athdf[f][slice_number,:,:])
            elif dimension == 'y':
                result[f].append(athdf[f][:, slice_number,:])
            else:
                result[f].append(athdf[f][:,:, slice_number])
                        
    return result

def animate_slice(data):
    plt.figure()
    for i in range(len(data)):
        plt.imshow(data[i])
        plt.title('Frame %d' % i)
        plt.show()
        plt.pause(0.2)
        clear_output(wait=True)




In [ ]:

data_file_name_template = data_folder+'/OrszagTang.out2.{}.athdf'

# this is time consuming, try do it once
data = read_all_timestep(data_file_name_template, num_timesteps, ['press', 'rho'], 0, 'x')



In [ ]:
# Cycle through the time steps and look at pressure
animate_slice(data['press'])

In [ ]:
# Now look at density
animate_slice(data['rho'])

## Slurm accounting

Slurm accounting has been moved to pcluster_cost_estimate notebook in the same folder. 

# Don't forget to clean up

1. Delete the ParallelCluster
2. Delete the RDS
3. S3 bucket
4. Secrets used in this excercise

Deleting VPC is risky, I will leave it out for you to manually clean it up if you created a new VPC. 

In [ ]:
print(pcluster_name, config_name)

In [ ]:
# this is used during developemnt, to reload the module after a change in the module
#try:
#    del sys.modules['pcluster_athena']
#except:
#    #ignore if the module is not loaded
#    print('Module not loaded, ignore')
    
#from pcluster_athena import PClusterHelper
# create the cluster - # You can rerun the rest of the notebook again with no harm. There are checks in place for existing resoources. 
pcluster_helper = PClusterHelper(pcluster_name, config_name, post_install_script_prefix)


!pcluster delete $pcluster_helper.pcluster_name

pcluster_helper.cleanup_after(KeepRDS=True)